In [3]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd

n = 200
data = np.zeros((n,9))
wt = np.zeros((n,1))
i = 0
rc_m = 201901
rc_y = 2019
while i < n:
  url = 'http://apis.data.go.kr/B551015/API4/raceResult'
  params ={'serviceKey' : 'D+m+QS+W9XDqddFT7tdxHDf4GTAO/HYy7scTAC45/+GnEjZJwk5QY2QDDxxJTOpRPWpBhfhrgAsEFD6dw9P9Ug==', 'pageNo' : '1', 'numOfRows' : '40', 'examdate' : '20220528', 'meet' : '1', 'rc_date' : '', 'rc_month' : str(rc_m), 'rc_no' : '1', 'rc_year' : str(rc_y) }
  response = requests.get(url,params=params)
  soup = BeautifulSoup(response.text, "lxml")
  items = soup.find_all("item")
  items
  for item in items:
    if item.find('rctime') == None:
      continue
    if item.find('rcdist') == None:
      continue
    if item.find('track') == None:
      continue
    if item.find('wghr') == None:
      continue
    if item.find('wgbudam') == None:
      continue
    if item.find('weather') == None:
      continue
    rctime = float(item.find('rctime').get_text())
    rcdist = float(item.find('rcdist').get_text())
    track = item.find('track').get_text()
    track = float(re.sub(r'[^0-9]', '', track))
    wghr = item.find('wghr').get_text()
    wghr = re.sub(r'([+-]+[0123456789]+)','', wghr)
    wghr = float(re.sub(r'[()]','',wghr))
    wgbudam = float(item.find('wgbudam').get_text())
    weather = item.find('weather').get_text()
    weather = re.sub('맑음', '1' ,weather)
    weather = re.sub('흐림', '0' ,weather)
    weather = re.sub('안개', '-1', weather)
    weather = re.sub('비', '-2', weather)
    weather = float(re.sub('눈','-3',weather))
    hrname = item.find('hrname').get_text()
    hrno = item.find('hrno').get_text()
    jkname = item.find('jkname').get_text()
    jkno = item.find('jkno').get_text()
    if rctime > 0 and rcdist > 0 and track > 0 and wghr > 0 and wgbudam > 0:
      print(i)

      url = 'http://apis.data.go.kr/B551015/API27/totalRecord'
      params ={'serviceKey' : 'D+m+QS+W9XDqddFT7tdxHDf4GTAO/HYy7scTAC45/+GnEjZJwk5QY2QDDxxJTOpRPWpBhfhrgAsEFD6dw9P9Ug==', 'pageNo' : '1', 'numOfRows' : '10', 'meet' : '1', 'pr_gubun' : '0', 'pr_name' : hrname, 'pr_no' : hrno }
      response2 = requests.get(url, params=params)
      soup2 = BeautifulSoup(response2.text, "lxml")
      item2 = soup2.find("item")
      if item2 == None:
        continue
      chaksunt = float(item2.find('chaksunt').get_text())
      rccntt = float(item2.find('rccntt').get_text())
      if rccntt == 0:
        data[i][5] = 0
      else:
        data[i][5] = chaksunt / rccntt

      url = 'http://apis.data.go.kr/B551015/API27/totalRecord'
      params ={'serviceKey' : 'D+m+QS+W9XDqddFT7tdxHDf4GTAO/HYy7scTAC45/+GnEjZJwk5QY2QDDxxJTOpRPWpBhfhrgAsEFD6dw9P9Ug==', 'pageNo' : '1', 'numOfRows' : '10', 'meet' : '1', 'pr_gubun' : '3', 'pr_name' : jkname, 'pr_no' : jkno }
      response3 = requests.get(url, params=params)
      soup3 = BeautifulSoup(response3.text, "lxml")
      item3 = soup3.find("item")
      if item3 == None:
        continue
      ord1cntt = float(item3.find('ord1cntt').get_text())
      ord2cntt = float(item3.find('ord2cntt').get_text())
      ord3cntt = float(item3.find('ord3cntt').get_text())
      rccntt = float(item3.find('rccntt').get_text())
      if rccntt == 0:
        data[i][6] = 0
        data[i][7] = 0
        data[i][8] = 0
      else:
        data[i][6] = ord1cntt / rccntt
        data[i][7] = ord2cntt / rccntt
        data[i][8] = ord3cntt / rccntt
      data[i][0] = rctime
      data[i][1] = rcdist
      data[i][2] = track
      data[i][3] = wghr
      data[i][4] = wgbudam
      wt[i][0] = weather


    else:
      continue
    i += 1
    if i >= n:
      break
  rc_m += 1
  if rc_m % 100 > 12:
    rc_m += 88
    rc_y += 1
  print(rc_m)

#x_data = np.delete(data,0,axis=1)
#y_data = np.delete(data,(1,2,3,4,5,6,7,8),axis=1)
#mean = np.sum(x_data, axis = 0) / n
#sqmean = x_data * x_data
#sqmean = np.sum(sqmean, axis = 0) / n
#dev = np.sqrt(sqmean - mean*mean)
#x_data = (x_data - mean) / dev
#data = np.concatenate((y_data,x_data),axis=1)
#data = np.concatenate((data, wt), axis = 1)
mean = np.sum(data, axis = 0) / n
dev = np.sum(data**2/n, axis = 0)
dev = np.sqrt(dev - mean**2)
data = (data - mean) / dev

print(mean)
print(dev)
print(data)
df = pd.DataFrame(np.array(data))
df.to_csv('test.csv', index=False)

0
1
2
3


KeyboardInterrupt: ignored